# Agent Architecture Patterns

This notebook explores different architectural patterns for building AI agents with Large Language Models (LLMs). We'll cover:

- Single-turn vs. multi-turn agents
- Tools vs. augmented prompt approaches
- Planning-focused architectures
- Action-focused architectures
- Hybrid approaches and when to use them

We'll implement examples of each pattern to understand their advantages, limitations, and best use cases.

## 1. Setup and Configuration

First, let's install the necessary libraries and set up our environment.

In [23]:
# Install required packages
!pip install python-dotenv requests tenacity

In [24]:
import os
import sys
import json
import time
import uuid
from typing import Dict, List, Any, Optional, Union
from dotenv import load_dotenv

# Add parent directory to path to import utility functions
sys.path.append('../../Week1/Day2')

# Import our API utilities
from api_utils import (
    call_openrouter,
    extract_text_response,
    extract_function_call,
    get_available_models
)

# Load environment variables
load_dotenv()

# Verify API key is loaded
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("✅ API key loaded successfully!")
    # Show first and last three characters for verification
    masked_key = f"{api_key[:3]}...{api_key[-3:]}" if len(api_key) > 6 else "[key too short]"
    print(f"API key: {masked_key}")
else:
    print("❌ API key not found! Make sure you've created a .env file with your OPENROUTER_API_KEY.")

✅ API key loaded successfully!
API key: sk-...843


## 2. Understanding Agent Architectures

Before diving into specific patterns, let's understand what an LLM agent is and the core components that make up an agent architecture.

### 2.1 What is an LLM Agent?

LLM-based agents (or LLM agents) are applications that can execute complex tasks through an architecture combining large language models with key modules like planning, memory, and tool usage. In these systems, the LLM serves as the main controller or "brain" that orchestrates the flow of operations needed to complete a task or user request.

Core components of agent architectures typically include:

1. **LLM Core**: The primary language model that processes inputs, generates outputs, and makes decisions.
2. **Memory Systems**: Both short-term (within a conversation) and long-term (across sessions) memory.
3. **Tools/Functions**: External capabilities the agent can use to interact with the world.
4. **Planning Module**: The ability to break down complex tasks into manageable steps.
5. **Reasoning Engine**: The ability to think through problems and make logical decisions.
6. **Action Executor**: The component that carries out actions in the external world.

Not all agents require all of these components, and the specific architecture you choose depends on your use case requirements.

## 3. Single-Turn vs. Multi-Turn Agents

One of the most fundamental distinctions in agent architectures is between single-turn and multi-turn agents.

### 3.1 Single-Turn Agents

Single-turn agents complete tasks with just one interaction between the user and the LLM. They receive a prompt, process it, and deliver the final output without any intermediate steps or additional interaction.

**Advantages:**
- Simplicity of implementation
- Lower latency (faster response time)
- Lower cost (single LLM call)
- Predictable behavior

**Limitations:**
- Limited ability to solve complex problems
- No feedback loop to correct misunderstandings
- No ability to access external tools or information beyond prompt
- Fixed context window limitations

Let's implement a simple single-turn agent:

In [4]:
def single_turn_agent(user_input: str, model: str = "openai/gpt-4o-mini-2024-07-18") -> str:
    """A simple single-turn agent that processes a user query and returns a response.
    
    Args:
        user_input: The user's query
        model: The model to use for processing
        
    Returns:
        The agent's response
    """
    # System instructions define the agent's behavior
    system_instructions = """
    You are a helpful AI assistant. Answer the user's question directly and concisely.
    If you don't know the answer, say so rather than making something up.
    """
    
    # Make a single API call
    response = call_openrouter(
        prompt=user_input,
        model=model,
        system_prompt=system_instructions,
        temperature=0.7,
        max_tokens=500
    )
    
    # Extract and return the text response
    if response.get("success", False):
        return extract_text_response(response)
    else:
        return f"Error: {response.get('error', 'Unknown error')}"

# Test the single-turn agent
query = "What are the three laws of robotics?"
result = single_turn_agent(query)
print(f"Query: {query}\n")
print(f"Response: {result}")

Query: What are the three laws of robotics?

Response: The Three Laws of Robotics, formulated by science fiction writer Isaac Asimov, are:

1. A robot may not injure a human being or, through inaction, allow a human being to come to harm.
2. A robot must obey the orders given to it by human beings, except where such orders would conflict with the First Law.
3. A robot must protect its own existence as long as such protection does not conflict with the First or Second Law.


### 3.2 Multi-Turn Agents

Multi-turn agents engage in an iterative process, with multiple interactions between the agent and its environment (which could include the user, tools, or external systems) before producing a final output.

**Advantages:**
- Can solve more complex problems through iteration
- Can leverage external tools and information sources
- Can correct misunderstandings through feedback
- Can adapt strategy based on intermediate results

**Limitations:**
- Higher latency (slower response time)
- Higher cost (multiple LLM calls)
- More complex implementation
- Potential for getting stuck in loops

Let's implement a basic multi-turn agent with conversation history:

In [5]:
def multi_turn_agent(user_input: str, conversation_history: List[Dict[str, str]] = None, model: str = "openai/gpt-4o-mini-2024-07-18") -> tuple:
    """A multi-turn agent that maintains conversation history.
    
    Args:
        user_input: The user's query
        conversation_history: List of previous message dictionaries
        model: The model to use for processing
        
    Returns:
        Tuple of (agent response, updated conversation history)
    """
    # Initialize conversation history if it doesn't exist
    if conversation_history is None:
        conversation_history = [
            {"role": "system", "content": """You are a helpful AI assistant. Maintain context throughout the conversation.
                Answer the user's questions directly and concisely. If you don't know the answer, say so rather than making something up.
                Use information from previous messages when relevant."""},
        ]
    
    # Add user input to conversation history
    conversation_history.append({"role": "user", "content": user_input})
    
    # Make API call with full conversation history
    response = call_openrouter(
        prompt=conversation_history,
        model=model,
        temperature=0.7,
        max_tokens=500
    )
    
    # Extract text response
    if response.get("success", False):
        assistant_response = extract_text_response(response)
        # Add assistant response to conversation history
        conversation_history.append({"role": "assistant", "content": assistant_response})
        return assistant_response, conversation_history
    else:
        error_message = f"Error: {response.get('error', 'Unknown error')}"
        return error_message, conversation_history

# Test the multi-turn agent with multiple interactions
# Initialize conversation
history = None

# First interaction
query1 = "What is machine learning?"
response1, history = multi_turn_agent(query1, history)

print(f"User: {query1}")
print(f"Agent: {response1}\n")

# Second interaction (referencing the first)
query2 = "What are some popular algorithms for it?"
response2, history = multi_turn_agent(query2, history)

print(f"User: {query2}")
print(f"Agent: {response2}")

User: What is machine learning?
Agent: Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. Instead, these systems learn from data by identifying patterns and making predictions or decisions based on that data. Machine learning is commonly used in applications such as image and speech recognition, natural language processing, and recommendation systems.

User: What are some popular algorithms for it?
Agent: Some popular machine learning algorithms include:

1. **Linear Regression**: Used for predicting a continuous target variable based on one or more predictors.

2. **Logistic Regression**: Used for binary classification problems to predict the probability of an outcome.

3. **Decision Trees**: A flowchart-like structure used for both classification and regression tasks.

4. **Random Forest**: An ensemble method that uses multiple 

## 4. Tools vs. Augmented Prompt Approaches

There are two main ways for LLM agents to extend their capabilities: using tools or augmenting prompts with external information.

### 4.1 Tool-Based Agents

Tool-based agents can call external functions or APIs to perform actions or retrieve information. These tools extend the agent's capabilities beyond what's possible with the LLM alone.

**Advantages:**
- Can interact with external systems and data sources
- Can perform real-world actions (like sending emails, booking appointments)
- Not limited by the LLM's knowledge cutoff
- Can handle structured data processing

**Limitations:**
- More complex to implement and maintain
- Requires careful security and permission management
- May introduce additional latency from external API calls
- Requires predefined tools for specific tasks

Let's define some tools and implement a basic tool-using agent:

In [22]:
# Define some simple tools
def get_weather(location: str) -> str:
    """Simulated weather lookup tool."""
    # In a real application, this would call a weather API
    weather_data = {
        "new york": "72°F, Partly Cloudy",
        "london": "61°F, Rainy",
        "tokyo": "78°F, Sunny",
        "sydney": "65°F, Windy"
    }
    return weather_data.get(location.lower(), "Weather data not available for this location")

def calculate(expression: str) -> str:
    """Simple calculator tool."""
    try:
        # Warning: eval can be dangerous in production environments
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# Define the tools in the format expected by the LLM
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name, e.g., 'New York', 'London'"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate",
        "description": "Perform a mathematical calculation",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate, e.g., '2 + 2', '3 * 4 / 2'"
                }
            },
            "required": ["expression"]
        }
    }
]

In [7]:
def tool_using_agent(user_input: str, conversation_history: List[Dict[str, str]] = None, model: str = "openai/gpt-4o-mini-2024-07-18") -> tuple:
    """An agent that can use tools to respond to user queries.
    
    Args:
        user_input: The user's query
        conversation_history: List of previous message dictionaries
        model: The model to use for processing
        
    Returns:
        Tuple of (agent response, updated conversation history)
    """
    # Initialize conversation history if it doesn't exist
    if conversation_history is None:
        conversation_history = [
            {"role": "system", "content": """You are a helpful AI assistant that can use tools to provide information.
                When you need specific information, use the appropriate tool. If a user's request can be answered
                using one of your tools, use the tool rather than providing a generic response.
                For weather information, use the get_weather tool.
                For calculations, use the calculate tool."""}
        ]
    
    # Add user input to conversation history
    conversation_history.append({"role": "user", "content": user_input})
    
    # Make API call with tools configuration
    response = call_openrouter(
        prompt=conversation_history,
        model=model,
        temperature=0.7,
        max_tokens=500,
        functions=tools  # Include our defined tools
    )
    
    if response.get("success", False):
        # Check if the model wants to call a function
        function_call = extract_function_call(response)
        
        if function_call.get("success", False):
            # The model wants to use a tool
            function_name = function_call.get("function_name")
            arguments = function_call.get("arguments", {})
            tool_id = function_call.get("tool_id", "call_" + str(uuid.uuid4()))
            
            # Execute the requested function
            if function_name == "get_weather":
                tool_result = get_weather(arguments.get("location", ""))
            elif function_name == "calculate":
                tool_result = calculate(arguments.get("expression", ""))
            else:
                tool_result = f"Error: Unknown function '{function_name}'"
                
            # Add the tool call and result to conversation history using the new format
            conversation_history.append({
                "role": "assistant",
                "content": None,
                "tool_calls": [
                    {
                        "id": tool_id,
                        "type": "function",
                        "function": {
                            "name": function_name,
                            "arguments": json.dumps(arguments)
                        }
                    }
                ]
            })
            
            conversation_history.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": tool_result
            })
            
            # Make a second API call to get the assistant's final response
            second_response = call_openrouter(
                prompt=conversation_history,
                model=model,
                temperature=0.7,
                max_tokens=500
            )
            
            if second_response.get("success", False):
                assistant_response = extract_text_response(second_response)
                conversation_history.append({"role": "assistant", "content": assistant_response})
                return assistant_response, conversation_history
            else:
                error_message = f"Error in second call: {second_response.get('error', 'Unknown error')}"
                return error_message, conversation_history
        else:
            # No function call, just a regular response
            assistant_response = extract_text_response(response)
            conversation_history.append({"role": "assistant", "content": assistant_response})
            return assistant_response, conversation_history
    else:
        error_message = f"Error: {response.get('error', 'Unknown error')}"
        return error_message, conversation_history

# Test the tool-using agent
# Try a weather query
weather_query = "What's the weather like in London?"
weather_response, history = tool_using_agent(weather_query)

print(f"User: {weather_query}")
print(f"Agent: {weather_response}\n")

# Try a calculation query
calc_query = "What's the result of 123 * 456?"
calc_response, history = tool_using_agent(calc_query, history)

print(f"User: {calc_query}")
print(f"Agent: {calc_response}\n")

# Try a general query that doesn't need tools
general_query = "What are some popular tourist attractions in Paris?"
general_response, history = tool_using_agent(general_query, history)

print(f"User: {general_query}")
print(f"Agent: {general_response}")

User: What's the weather like in London?
Agent: The weather in London is currently 61°F and rainy.

User: What's the result of 123 * 456?
Agent: The result of 123 * 456 is 56,088.

User: What are some popular tourist attractions in Paris?
Agent: Some popular tourist attractions in Paris include:

1. **Eiffel Tower** - An iconic symbol of Paris, offering stunning views of the city.
2. **Louvre Museum** - The world's largest art museum, home to thousands of works including the Mona Lisa.
3. **Notre-Dame Cathedral** - A masterpiece of French Gothic architecture, located on the Île de la Cité.
4. **Champs-Élysées and Arc de Triomphe** - A famous avenue leading to the monumental arch honoring those who fought for France.
5. **Montmartre and Sacré-Cœur Basilica** - A historic district known for its artistic history and the stunning basilica at its summit.
6. **Seine River Cruises** - Offering a unique perspective of the city's landmarks along the river.
7. **Palace of Versailles** - A short 

### 4.2 Augmented Prompt Approaches (RAG)

Retrieval-Augmented Generation (RAG) is an alternative approach that enhances LLM capabilities by retrieving relevant information and including it in the prompt, rather than using external tools.

**Advantages:**
- Simpler implementation (no complex tool interaction logic)
- Can be done in a single LLM call (potentially lower latency)
- Can incorporate domain-specific knowledge without custom tool development
- More flexible and adaptable to new information sources

**Limitations:**
- Limited by context window size
- Cannot perform actions, only retrieve information
- Information retrieval quality depends on embedding and retrieval methods
- May lead to inconsistent results based on retrieval quality

Let's implement a simple RAG agent:

In [8]:
# Create a simple knowledge base for demonstration
knowledge_base = {
    "company_info": """
    TechCorp was founded in 2010 by Jane Smith and John Doe.
    Headquarters: San Francisco, CA
    Number of employees: 1,200
    Annual revenue: $250 million
    Main products: Cloud software, AI solutions, and data analytics tools
    CEO: Dr. Jane Smith
    CTO: Dr. John Doe
    """,
    
    "product_info": """
    TechCloud: Our flagship cloud computing platform
    - Price: Starting at $99/month for business tier
    - Features: Unlimited storage, 99.9% uptime guarantee, 24/7 support
    - Released: 2015, latest version 4.2 (2023)
    
    DataInsight: Enterprise data analytics solution
    - Price: $499/month per 10 users
    - Features: Real-time analytics, custom dashboards, ML-powered predictions
    - Released: 2018, latest version 2.5 (2023)
    
    AI Assistant: Conversational AI platform for businesses
    - Price: $199/month base + $10 per 1000 queries
    - Features: Natural language processing, integration with 100+ business tools
    - Released: 2020, latest version 1.8 (2023)
    """,
    
    "customer_support": """
    Support hours: 24/7 for Enterprise plans, 8am-8pm EST for other plans
    Contact methods:
    - Email: support@techcorp.example
    - Phone: 1-800-TECH-123
    - Live chat: Available on website
    
    Average response time: 2 hours for Standard, 30 minutes for Premium, 10 minutes for Enterprise
    Refund policy: 30-day money-back guarantee for all products
    """
}

In [9]:
def simple_retriever(query: str, knowledge_base: Dict[str, str]) -> List[str]:
    """A simple keyword-based retriever for demonstration purposes.
    
    In a real-world application, this would be replaced with a proper vector database
    and embedding-based semantic search.
    
    Args:
        query: The user's question
        knowledge_base: Dictionary of knowledge chunks
        
    Returns:
        List of relevant knowledge chunks
    """
    query_lower = query.lower()
    results = []
    
    # Simple keyword matching
    keywords = {
        "company_info": ["founder", "founded", "headquarters", "employees", "revenue", "ceo", "cto", "company"],
        "product_info": ["product", "price", "features", "techcloud", "datainsight", "ai assistant", "cost", "version"],
        "customer_support": ["support", "contact", "email", "phone", "chat", "hours", "response time", "refund"]
    }
    
    # Check for keyword matches
    for category, words in keywords.items():
        if any(word in query_lower for word in words):
            results.append(knowledge_base[category])
    
    # If no specific matches, return all knowledge
    if not results:
        results = list(knowledge_base.values())
        
    return results

def rag_agent(user_input: str, knowledge_base: Dict[str, str], model: str = "openai/gpt-4o-mini-2024-07-18") -> str:
    """A simple RAG agent that retrieves information and augments the prompt.
    
    Args:
        user_input: The user's query
        knowledge_base: The knowledge base to retrieve from
        model: The model to use
        
    Returns:
        The agent's response
    """
    # Retrieve relevant information
    retrieved_info = simple_retriever(user_input, knowledge_base)
    
    # Construct the augmented prompt
    system_prompt = """
    You are a company information assistant for TechCorp. Answer the user's question 
    based only on the provided information. If the answer is not in the information provided,
    say that you don't have that information, but don't make up an answer.
    """
    
    # Add retrieved information to the system prompt
    system_prompt += "\n\nInformation:\n" + "\n\n".join(retrieved_info)
    
    # Make the API call
    response = call_openrouter(
        prompt=user_input,
        model=model,
        system_prompt=system_prompt,
        temperature=0.3,  # Lower temperature for more factual responses
        max_tokens=500
    )
    
    # Extract and return the text response
    if response.get("success", False):
        return extract_text_response(response)
    else:
        return f"Error: {response.get('error', 'Unknown error')}"

# Test the RAG agent with some queries
company_query = "Who founded TechCorp and when?"
product_query = "What is DataInsight and how much does it cost?"
support_query = "How can I contact customer support?"

print(f"Query: {company_query}")
print(f"Response: {rag_agent(company_query, knowledge_base)}\n")

print(f"Query: {product_query}")
print(f"Response: {rag_agent(product_query, knowledge_base)}\n")

print(f"Query: {support_query}")
print(f"Response: {rag_agent(support_query, knowledge_base)}")

Query: Who founded TechCorp and when?
Response: TechCorp was founded in 2010 by Jane Smith and John Doe.

Query: What is DataInsight and how much does it cost?
Response: DataInsight is an enterprise data analytics solution that offers real-time analytics, custom dashboards, and ML-powered predictions. The price is $499 per month for every 10 users.

Query: How can I contact customer support?
Response: You can contact customer support through the following methods:

- Email: support@techcorp.example
- Phone: 1-800-TECH-123
- Live chat: Available on the website


## 5. Planning-Focused vs. Action-Focused Architectures

Now let's explore the distinction between architectures that emphasize planning versus those that focus on actions.

### 5.1 Planning-Focused Architectures (Plan-and-Execute)

Planning-focused architectures follow a "plan-and-execute" pattern. They first create a comprehensive plan with defined steps, then execute each step to complete the task.

**Advantages:**
- More efficient for complex, multi-step tasks
- Better overall task completion through explicit planning
- Reduces the number of LLM calls for executing steps
- Allows for human review of plans before execution

**Limitations:**
- Less adaptable to unexpected situations
- Initial planning step may miss important details
- Plan quality heavily dependent on initial prompt quality
- May struggle with tasks that require dynamic adjustment

Let's implement a simple planning-focused agent:

In [14]:
def planning_agent(user_input: str, model: str = "openai/gpt-4o-mini-2024-07-18") -> Dict[str, Any]:
    """A planning-focused agent that creates a plan first, then executes it.
    
    Args:
        user_input: The user's task
        model: The model to use
        
    Returns:
        Dictionary with plan, execution results, and final output
    """
    # Step 1: Generate a plan
    planning_prompt = f"""Task: {user_input}
    
    Create a detailed step-by-step plan to accomplish this task. Each step should be clear and actionable.
    Format the plan as a numbered list. Include 3-7 steps depending on the complexity of the task.
    """
    
    planning_system_prompt = """
    You are a planning AI that creates detailed, logical plans to accomplish tasks.
    Break complex tasks into clear, actionable steps that can be followed sequentially.
    Focus only on creating the plan, not executing it.
    """
    
    planning_response = call_openrouter(
        prompt=planning_prompt,
        model=model,
        system_prompt=planning_system_prompt,
        temperature=0.3,
        max_tokens=500
    )
    
    if not planning_response.get("success", False):
        return {"error": f"Planning failed: {planning_response.get('error', 'Unknown error')}"}
    
    plan = extract_text_response(planning_response)
    
    # Step 2: Execute each step in the plan
    # For demonstration, we'll simulate execution by having the LLM explain how it would execute each step
    execution_results = []
    
    # Split the plan into steps (assuming it's a numbered list)
    steps = []
    for line in plan.strip().split('\n'):
        line = line.strip()
        if line and (line[0].isdigit() or (len(line) > 2 and line[0:2] in ['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.'])):
            steps.append(line)
    
    # If we couldn't parse numbered steps, treat the whole plan as one step
    if not steps:
        steps = [plan]
    
    for i, step in enumerate(steps):
        execution_prompt = f"""Task: {user_input}
        
        Overall plan:
        {plan}
        
        Current step to execute: {step}
        
        Explain in detail how you would execute this specific step. Provide the actual execution,
        not just a theoretical explanation. If research or information gathering is needed,
        provide realistic results.
        """
        
        execution_system_prompt = """
        You are an execution AI that carries out specific steps in a plan.
        Focus only on executing the current step. Be thorough and detailed.
        Provide concrete results, not just explanations of how you would approach it.
        """
        
        execution_response = call_openrouter(
            prompt=execution_prompt,
            model=model,
            system_prompt=execution_system_prompt,
            temperature=0.7,
            max_tokens=500
        )
        
        if not execution_response.get("success", False):
            execution_results.append(f"Error executing step {i+1}: {execution_response.get('error', 'Unknown error')}")
        else:
            execution_results.append(extract_text_response(execution_response))
    
    # Step 3: Generate a final summary
    summary_prompt = f"""Task: {user_input}
    
    Plan:
    {plan}
    
    Execution results:
    {"..".join([f"Step {i+1}: {result}" for i, result in enumerate(execution_results)])}
    
    Please provide a concise summary of the completed task, highlighting key findings or outcomes.
    """
    
    summary_system_prompt = """
    You are a summarization AI that synthesizes results from a multi-step process.
    Provide a clear, concise summary that highlights the most important points and outcomes.
    Focus on answering the original task/question directly.
    """
    
    summary_response = call_openrouter(
        prompt=summary_prompt,
        model=model,
        system_prompt=summary_system_prompt,
        temperature=0.5,
        max_tokens=500
    )
    
    if not summary_response.get("success", False):
        final_summary = f"Error generating summary: {summary_response.get('error', 'Unknown error')}"
    else:
        final_summary = extract_text_response(summary_response)
    
    return {
        "plan": plan,
        "execution_results": execution_results,
        "summary": final_summary
    }

# Test the planning agent with a complex task
complex_task = "Analyze the pros and cons of remote work for both employees and employers"
planning_result = planning_agent(complex_task)

print(f"Task: {complex_task}\n")
print("Generated Plan:")
print(planning_result["plan"])
print("\nExecution Results:")
for i, result in enumerate(planning_result["execution_results"]):
    print(f"\nStep {i+1} Execution:")
    print(result[:200] + "..." if len(result) > 200 else result)
print("\nFinal Summary:")
print(planning_result["summary"])

Task: Analyze the pros and cons of remote work for both employees and employers

Generated Plan:
1. **Define the Scope of Analysis**  
   - Identify the key aspects of remote work to be analyzed for both employees and employers. This may include productivity, work-life balance, communication, cost savings, and employee satisfaction.

2. **Conduct Literature Review**  
   - Research existing studies, articles, and reports on remote work. Focus on both qualitative and quantitative data that highlight the pros and cons for employees and employers. Take notes on relevant findings.

3. **Create a Pros and Cons Framework**  
   - Develop a structured framework to categorize the pros and cons for both employees and employers. This could be a simple table with four quadrants: Employee Pros, Employee Cons, Employer Pros, and Employer Cons.

4. **Gather Employee Perspectives**  
   - Design and distribute a survey or conduct interviews with employees who have experience with remote work. Ask spe

### 5.2 Action-Focused Architectures (ReAct)

ReAct (Reasoning+Acting) is an action-focused architecture that interleaves reasoning and action steps. It follows a "think-act-observe" loop, making decisions dynamically based on the current state.

**Advantages:**
- More adaptable to changing conditions
- Better handling of unexpected situations
- Can explore and gather information dynamically
- Good for tasks with uncertain paths or requirements

**Limitations:**
- Higher latency due to multiple LLM calls
- May lead to inefficient exploration
- Risk of getting stuck in loops
- More complex to implement and debug

Let's implement a simple ReAct agent:

In [15]:
def react_agent(user_input: str, tools_dict: Dict[str, callable], model: str = "openai/gpt-4o-mini-2024-07-18", max_steps: int = 5) -> Dict[str, Any]:
    """A ReAct agent that follows the Reasoning+Acting pattern.
    
    Args:
        user_input: The user's task
        tools_dict: Dictionary mapping tool names to functions
        model: The model to use
        max_steps: Maximum number of iterations
        
    Returns:
        Dictionary with thinking steps, action history, and final answer
    """
    # Convert tools to the format expected by the LLM
    tools_spec = []
    for tool_name, tool_fn in tools_dict.items():
        # Get parameter list from function signature
        import inspect
        signature = inspect.signature(tool_fn)
        params = {}
        required = []
        
        for param_name, param in signature.parameters.items():
            params[param_name] = {
                "type": "string",
                "description": f"The {param_name} parameter for {tool_name}"
            }
            if param.default == inspect.Parameter.empty:
                required.append(param_name)
        
        tools_spec.append({
            "name": tool_name,
            "description": tool_fn.__doc__ or f"The {tool_name} tool",
            "parameters": {
                "type": "object",
                "properties": params,
                "required": required
            }
        })
    
    # Initialize the conversation
    conversation_history = [
        {"role": "system", "content": """You are a problem-solving AI that follows the ReAct (Reasoning + Acting) framework.
            For each step, you'll think about what needs to be done, choose an action, and then observe the result.
            
            Follow this format for each step:
            Thought: <your reasoning about what to do next>
            Action: <tool name to use>
            
            After seeing the observation, continue with your next thought.
            When you have enough information to provide a final answer, use:
            Thought: I now know the final answer
            Final Answer: <your detailed answer to the original question>
            
            Always start with a Thought."""}
    ]
    
    # Add the user's task
    conversation_history.append({"role": "user", "content": user_input})
    
    # Initialize tracking
    thinking_steps = []
    action_history = []
    final_answer = ""
    
    # ReAct loop
    for step in range(max_steps):
        # Get the next thought and action from the LLM
        response = call_openrouter(
            prompt=conversation_history,
            model=model,
            temperature=0.7,
            max_tokens=500,
            functions=tools_spec
        )
        
        if not response.get("success", False):
            return {"error": f"Error in step {step+1}: {response.get('error', 'Unknown error')}"}
        
        # Get the assistant's response
        assistant_response = extract_text_response(response)
        thinking_steps.append(assistant_response)
        
        # Check if we've reached a final answer
        if "Final Answer:" in assistant_response:
            # Extract the final answer
            final_answer = assistant_response.split("Final Answer:")[1].strip()
            conversation_history.append({"role": "assistant", "content": assistant_response})
            break
        
        # Check for function calls
        function_call = extract_function_call(response)
        
        if function_call.get("success", False):
            # Execute the function call
            function_name = function_call.get("function_name")
            arguments = function_call.get("arguments", {})
            tool_id = function_call.get("tool_id", "call_" + str(uuid.uuid4()))
            
            if function_name in tools_dict:
                try:
                    tool_result = tools_dict[function_name](**arguments)
                    action_history.append({"tool": function_name, "args": arguments, "result": tool_result})
                except Exception as e:
                    tool_result = f"Error executing {function_name}: {str(e)}"
                    action_history.append({"tool": function_name, "args": arguments, "error": str(e)})
            else:
                tool_result = f"Error: Unknown tool '{function_name}'"
                action_history.append({"tool": function_name, "args": arguments, "error": "Unknown tool"})
            
            # Add the tool call and result to conversation using the new format
            conversation_history.append({
                "role": "assistant",
                "content": assistant_response,
                "tool_calls": [
                    {
                        "id": tool_id,
                        "type": "function",
                        "function": {
                            "name": function_name,
                            "arguments": json.dumps(arguments)
                        }
                    }
                ]
            })
            
            conversation_history.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": str(tool_result)
            })
        else:
            # No function call, just thinking
            conversation_history.append({"role": "assistant", "content": assistant_response})
            # Add a prompt to nudge the agent to continue with the ReAct pattern
            conversation_history.append({"role": "user", "content": "Continue with your next thought and action."})
    
    # Check if we reached max steps without a final answer
    if not final_answer:
        # Ask for a final answer
        final_prompt = "You've reached the maximum number of steps. Please provide your final answer now based on what you've learned."
        conversation_history.append({"role": "user", "content": final_prompt})
        
        final_response = call_openrouter(
            prompt=conversation_history,
            model=model,
            temperature=0.7,
            max_tokens=500
        )
        
        if final_response.get("success", False):
            final_answer = extract_text_response(final_response)
        else:
            final_answer = f"Error getting final answer: {final_response.get('error', 'Unknown error')}"
    
    return {
        "thinking_steps": thinking_steps,
        "action_history": action_history,
        "final_answer": final_answer
    }

# Define additional tools for the ReAct agent
def search_web(query: str) -> str:
    """Simulated web search tool."""
    # In a real application, this would call a search API
    search_results = {
        "remote work": "Recent studies show 70% of companies plan to adopt hybrid work models post-pandemic. Benefits include reduced office costs and wider talent pools. Challenges include team cohesion and maintaining company culture.",
        "employee satisfaction": "84% of remote workers report increased job satisfaction according to a 2023 survey. Key factors include better work-life balance and eliminated commute time.",
        "productivity": "Studies on remote work productivity show mixed results. Some report up to 13% productivity gains, while others note 10-15% decreases, particularly for collaborative tasks.",
        "employer benefits": "Employers report 22% reduction in turnover, 30% reduction in real estate costs, and access to 70% larger talent pools with remote work policies.",
        "challenges": "Top challenges of remote work include: communication difficulties (63%), isolation/loneliness (59%), difficulty separating work/home life (47%), and managing remote teams (44%)."
    }
    
    # Find the closest matching query
    query_lower = query.lower()
    for key, value in search_results.items():
        if key in query_lower:
            return f"Search results for '{query}': {value}"
    
    return f"No specific results found for '{query}'. Try a more specific search term."

def get_statistics(topic: str) -> str:
    """Simulated statistics lookup tool."""
    stats = {
        "remote work": """
        - 16% of companies globally are fully remote (Owl Labs, 2023)
        - 98% of workers want to work remotely at least some of the time (Buffer, 2023)
        - 77% of remote workers report higher productivity (Stanford, 2022)
        - 55% of businesses globally offer some capacity for remote work (Gartner, 2023)
        """,
        
        "employee satisfaction": """
        - Remote workers are 22% happier than workers in an office setting (Tracking Happiness, 2023)
        - 74% of workers say that having remote options would make them less likely to leave a company (Owl Labs, 2023)
        - Remote workers save 40-60 minutes per day without commuting (Upwork, 2022)
        """,
        
        "employer benefits": """
        - Companies save an average of $11,000 per year per employee who works remotely half-time (Global Workplace Analytics, 2023)
        - Remote companies see 25% less turnover than companies without remote options (Owl Labs, 2023)
        - 63% of high-growth companies use a "productivity anywhere" workforce model (Accenture, 2022)
        """,
        
        "challenges": """
        - 60% of remote workers report feeling less connected to their teams (Buffer, 2023)
        - 45% of remote workers report working more hours than they did in the office (Owl Labs, 2022)
        - 32% of managers say they struggle with managing the performance of remote workers (Harvard Business Review, 2023)
        - 27% of remote workers have experienced feelings of isolation or loneliness (Buffer, 2023)
        """
    }
    
    # Find the closest matching topic
    topic_lower = topic.lower()
    for key, value in stats.items():
        if key in topic_lower:
            return f"Statistics on {topic}: {value}"
    
    return f"No statistics found for '{topic}'. Available topics: remote work, employee satisfaction, employer benefits, challenges."

# Create a tools dictionary
tools = {
    "get_weather": get_weather,
    "calculate": calculate,
    "search_web": search_web,
    "get_statistics": get_statistics
}

# Test the ReAct agent
react_task = "Analyze the pros and cons of remote work for both employees and employers"
react_result = react_agent(react_task, tools)

print(f"Task: {react_task}\n")
print("ReAct Thinking Steps:")
for i, step in enumerate(react_result["thinking_steps"]):
    print(f"\nStep {i+1}:")
    print(step[:200] + "..." if len(step) > 200 else step)

print("\nTool Usage History:")
for i, action in enumerate(react_result["action_history"]):
    print(f"\nAction {i+1}: Used {action['tool']} with args {action['args']}")
    if "result" in action:
        print(f"Result: {action['result'][:100]}..." if len(action['result']) > 100 else f"Result: {action['result']}")
    else:
        print(f"Error: {action['error']}")

print("\nFinal Answer:")
print(react_result["final_answer"])

Task: Analyze the pros and cons of remote work for both employees and employers

ReAct Thinking Steps:

Step 1:
Thought: To provide a comprehensive analysis of the pros and cons of remote work for both employees and employers, I will brainstorm and categorize the benefits and drawbacks for both sides. 

Action:...

Tool Usage History:

Final Answer:
Remote work has distinct pros and cons for both employees and employers:

### For Employees:
**Pros:**
- Flexibility in schedule and location.
- Cost savings on commuting and work-related expenses.
- Better work-life balance.
- Increased productivity due to fewer distractions.

**Cons:**
- Feelings of isolation and disconnection from colleagues.
-


## 6. Hybrid Approaches and When To Use Them

In practice, many effective agent architectures combine elements from different patterns to leverage their respective strengths. Let's explore some hybrid approaches.

### 6.1 Plan-Then-ReAct Hybrid

This hybrid approach starts with a planning phase to create a high-level strategy, then uses ReAct for dynamic execution of each step. This combines the strategic overview of planning-focused approaches with the adaptability of action-focused ones.

**Best Use Cases:**
- Complex tasks with uncertain subtasks
- Tasks requiring both structure and flexibility
- Information-gathering tasks with an overall goal
- Research and analysis projects

Let's implement a simple Plan-Then-ReAct hybrid agent:

In [20]:
def plan_then_react_agent(user_input: str, tools_dict: Dict[str, callable], model: str = "openai/gpt-4o-mini-2024-07-18") -> Dict[str, Any]:
    """A hybrid agent that first plans, then uses ReAct for execution.
    
    Args:
        user_input: The user's task
        tools_dict: Dictionary mapping tool names to functions
        model: The model to use
        
    Returns:
        Dictionary with plan, execution history, and final answer
    """
    # Step 1: Generate a plan
    planning_prompt = f"""Task: {user_input}
    
    Create a high-level plan to accomplish this task. Each step should be clear and actionable.
    Focus on what information needs to be gathered and what analyses need to be performed.
    Format the plan as a numbered list with 3-5 main steps.
    """
    
    planning_system_prompt = """
    You are a planning AI that creates strategic plans to accomplish tasks.
    Break complex tasks into high-level steps that guide the overall approach.
    Focus on the overall strategy, not detailed execution.
    """
    
    planning_response = call_openrouter(
        prompt=planning_prompt,
        model=model,
        system_prompt=planning_system_prompt,
        temperature=0.3,
        max_tokens=500
    )
    
    if not planning_response.get("success", False):
        return {"error": f"Planning failed: {planning_response.get('error', 'Unknown error')}"}
    
    plan = extract_text_response(planning_response)
    
    # Step 2: Execute each high-level step using ReAct
    # Split the plan into steps (assuming it's a numbered list)
    steps = []
    for line in plan.strip().split('\n'):
        line = line.strip()
        if line and (line[0].isdigit() or (len(line) > 2 and line[0:2] in ['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.'])):
            steps.append(line)
    
    # If we couldn't parse numbered steps, treat the whole plan as one step
    if not steps:
        steps = [plan]
    
    step_executions = []
    all_thinking_steps = []
    all_action_history = []
    
    for i, step in enumerate(steps):
        step_prompt = f"""
        
        Overall Task: {user_input}
        
        Overall Plan:
        {plan}
        
        Current Step to Execute: {step}
        
        Execute this specific step using the available tools. Gather information, perform analysis,
        and provide detailed results for this step.
        """
        
        # Use ReAct for this specific step
        react_result = react_agent(step_prompt, tools_dict, model, max_steps=3)
        
        if "error" in react_result:
            step_executions.append({"step": step, "error": react_result["error"]})
        else:
            step_executions.append({
                "step": step,
                "thinking": react_result["thinking_steps"],
                "actions": react_result["action_history"],
                "result": react_result["final_answer"]
            })
            all_thinking_steps.extend(react_result["thinking_steps"])
            all_action_history.extend(react_result["action_history"])
    
    # Step 3: Generate a final summary
    summary_prompt = f"""Task: {user_input}
    
    Plan:
    {plan}
    
    Execution results:
    {"..".join([f"Step {i+1}: {exec_result.get('result', exec_result.get('error', 'Unknown error'))}" for i, exec_result in enumerate(step_executions)])}
    
    Please provide a comprehensive answer to the original task, synthesizing all the information collected.
    Be detailed and thorough.
    """
    
    summary_system_prompt = """
    You are a synthesis AI that consolidates information from multiple sources.
    Provide a comprehensive, cohesive answer that addresses all aspects of the original task.
    Include specific details and insights from the execution results.
    """
    
    summary_response = call_openrouter(
        prompt=summary_prompt,
        model=model,
        system_prompt=summary_system_prompt,
        temperature=0.5,
        max_tokens=800
    )
    
    if not summary_response.get("success", False):
        final_summary = f"Error generating summary: {summary_response.get('error', 'Unknown error')}"
    else:
        final_summary = extract_text_response(summary_response)
    
    return {
        "plan": plan,
        "step_executions": step_executions,
        "all_thinking_steps": all_thinking_steps,
        "all_action_history": all_action_history,
        "final_answer": final_summary
    }

# Test the hybrid agent
hybrid_task = "Research the impact of artificial intelligence on healthcare, including benefits and challenges"
hybrid_result = plan_then_react_agent(hybrid_task, tools)

print(f"Task: {hybrid_task}\n")
print("Generated Plan:")
print(hybrid_result["plan"])

print("\nExecution Summary:")
for i, step_exec in enumerate(hybrid_result["step_executions"]):
    print(f"\nStep {i+1}: {step_exec['step']}")
    if "result" in step_exec:
        print(f"Result: {step_exec['result'][:100]}..." if len(step_exec['result']) > 100 else f"Result: {step_exec['result']}")
    else:
        print(f"Error: {step_exec['error']}")

print("\nFinal Answer:")
print(hybrid_result["final_answer"][:500] + "..." if len(hybrid_result["final_answer"]) > 500 else hybrid_result["final_answer"])

Task: Research the impact of artificial intelligence on healthcare, including benefits and challenges

Generated Plan:
1. **Define Research Objectives and Scope**  
   - Identify specific areas of healthcare impacted by AI (e.g., diagnostics, treatment planning, patient management).
   - Determine the key benefits and challenges to be explored (e.g., efficiency, accuracy, ethical concerns, data privacy).

2. **Conduct Literature Review**  
   - Gather existing research papers, articles, and case studies on AI applications in healthcare.
   - Summarize findings on both the benefits and challenges of AI integration in healthcare settings.

3. **Identify Key Stakeholders and Experts**  
   - List relevant stakeholders in the healthcare and AI sectors (e.g., healthcare providers, technology developers, regulatory bodies).
   - Plan outreach to experts for interviews or surveys to gain insights on real-world applications and challenges.

4. **Analyze Collected Data**  
   - Synthesize infor

### 6.2 When to Use Different Architectures

Choosing the right agent architecture depends on the specific task requirements and constraints. Here's a guide for when to use each approach:

#### Single-Turn Agents
- **Best for**: Simple queries, factual questions, straightforward tasks
- **When to use**: Low latency is critical, costs need to be minimized, user queries are typically simple
- **Example use cases**: FAQ answering, simple content generation, style transfer, summarization

#### Multi-Turn Agents
- **Best for**: Conversations, iterative tasks, tasks requiring clarification
- **When to use**: User interaction is important, task clarity emerges through dialogue
- **Example use cases**: Customer support, tutoring, interviewing, coaching

#### Tool-Based Agents
- **Best for**: Tasks requiring external actions or data retrieval
- **When to use**: When LLM knowledge is insufficient, real-world actions are needed
- **Example use cases**: Booking systems, data analysis, workflow automation, personal assistants

#### RAG-Based Agents
- **Best for**: Knowledge-intensive tasks with domain-specific information
- **When to use**: When information is available but not in the LLM's training data
- **Example use cases**: Legal assistance, medical information, technical support, research

#### Planning-Focused Agents
- **Best for**: Complex tasks with clear steps, predictable workflows
- **When to use**: When efficiency matters, steps are known in advance, human oversight is available
- **Example use cases**: Project planning, document writing, analysis reports, code generation

#### Action-Focused (ReAct) Agents
- **Best for**: Exploratory tasks, unpredictable workflows, information gathering
- **When to use**: When adaptability matters more than efficiency, dynamic exploration is needed
- **Example use cases**: Research, troubleshooting, creative problem-solving, open-ended exploration

#### Hybrid Agents
- **Best for**: Complex tasks with both structured and unpredictable elements
- **When to use**: When neither planning-only nor react-only approaches are sufficient
- **Example use cases**: Comprehensive research projects, product development, strategic planning

## 7. Conclusion and Best Practices

In this notebook, we've explored various agent architecture patterns for building LLM-based systems. Each pattern has its own strengths, limitations, and ideal use cases.

### Key Takeaways

1. **Match architecture to task requirements**: Consider the nature of the task, expected user interactions, and system constraints when choosing an architecture.

2. **Start simple and scale up**: Begin with the simplest architecture that meets your needs, then add complexity only when necessary.

3. **Consider hybrid approaches**: Many real-world applications benefit from combining elements of different architectural patterns.

4. **Evaluate empirically**: Test different architectures with real users and measure performance on key metrics.

5. **Balance cost and performance**: More complex architectures often require more LLM calls, increasing latency and cost.

### Best Practices

1. **Clear system prompts**: Regardless of architecture, provide clear, detailed instructions to the LLM.

2. **Error handling**: Implement robust error handling for all components, especially for tool calls.

3. **User feedback**: Create mechanisms for users to provide feedback when agent responses miss the mark.

4. **Continuous improvement**: Collect and analyze interaction logs to identify failure modes and improvement opportunities.

5. **Responsible design**: Implement safeguards against misuse, bias, and harmful outputs.

6. **Transparency**: Make the agent's capabilities and limitations clear to users to set appropriate expectations.

By understanding and applying these different architectural patterns, you can build more effective, reliable, and user-friendly LLM-based agents for a wide range of applications.